# Concept or Idea Generation (Notebook Only, No Service)

This demo show how to generate highly-visual ideas, called concepts, from Text2Image LLMs using various text-based inputs. This use case is just one way a concept could come to life. 

**Important Note:** This Notebook provides the end-to-end demo, but can only be run within this Notebook. In order to create a persistent model that can be callable from other places (e.g. Streamlit, Self-hosted Application, API, etc), please use the other Notebook.


## Prerequisites

#### Role and Permissions
You cannot use ACCOUNTADMIN roles. Follow Step 1 in `1_set_up.sql`

#### Container Runtime Notebook
Select Notebook Setting in the top-right and select "Run on Container"

#### External Access Integrations for Pypi and Huggingface
Run the Step 3 section in `1_set_up.sql`

## General Demo Flow
End goal - Generate an image from a description of a new product or an advertising brief. Apply a brand logo or product image as well.

#### Steps:
1. Create a Model Pipeline - _Prime the model to generate the images we need_
2. Describe Our Concept - _Link any needed logo/product images and describe our idea_
3. Generate the Concept - _Create the image to showcase the concept_
4. Merge Images - _Annotate our concept with our brand logo or product image_
5. Add Marketing Tagline - _An optional step if you want to generate any marketing copy_

_Note: This demo was inspired by a sentence in a recent [article](https://consumergoods.com/pg-taps-ai-and-automation-faster-smarter-operations) about P&G leveraging GenAI to develop concept ideas from various data sources._

# STEP 1: Install third-party packages and create a model Pipeline

In [ ]:
# Install the diffusers package that will allow us to download models from huggingfacehub and create model pipelines.
# NOTE: If there is a warning below that says `External Access Required`, you need to click on the menu in the top-right,
#       then select Notebook settings, select the external access tab, and turn on both the pypi EAI and the huggingface EAI

!pip install diffusers

In [ ]:
# Install other required packages and start a snowpark session
import json
import torch
import requests
import numpy as np
import streamlit as st
from io import BytesIO
from PIL import Image, ImageDraw, ImageFont
from diffusers import AutoPipelineForText2Image
from snowflake.cortex import Complete
from snowflake.snowpark.context import get_active_session

# Get the current credentials for a session
session = get_active_session()

In [ ]:
# Download the model from HuggingFace Hub and create a pipeline
# NOTE: This step will take time because of the model download (often 20+ GB)

# Select your image model - models have different levels of quality/size, FLUX.1-Schnell is recommended
IMAGE_MODEL = 'black-forest-labs/FLUX.1-schnell'
#IMAGE_MODEL = 'stabilityai/stable-diffusion-xl-base-1.0'

pipeline = AutoPipelineForText2Image.from_pretrained(
    IMAGE_MODEL, 
    torch_dtype=torch.float16,
)

In [ ]:
# If you are running on GPU_NV_L or have enough VRAM, then use this
#pipeline.to('cuda')

# If you are running on GPU_NV_M, use this to minimize VRAM requirements
pipeline.enable_sequential_cpu_offload()

# STEP 2: Describe our Concept

### Method 1: From User Input or External Data Sources

In [ ]:
# Bring in a brand logo or product image from a URL
# From URL
img_url = 'https://raw.githubusercontent.com/sfc-gh-pnanisetty/concept-generator-service/refs/heads/main/charisma_paper_towels.png' 
img_logo = Image.open(requests.get(img_url, stream=True).raw)

In [ ]:
# Provide the description of the brand being used and a concept idea
brand_description = 'Charisma paper towels'
concept_idea = 'a paper towel roll with christmas-style designs on the paper'

### Method 2: From Data Input (Customer reviews, documents, etc)

In [ ]:
# Bring in a brand logo or product image from a Snowflake Stage
# Replace with the name of your file at the end
img_raw = session.file.get_stream("@CONCEPT_GEN_DB.CONCEPT_GEN_SCHEMA.CONCEPT_GEN_INPUT_IMAGES/charisma_paper_towels.png" , decompress=False).read()
img_logo = Image.open(BytesIO(img_raw))

In [ ]:
# Load a single idea from a table of input ideas
idea_df = session.table('CONCEPT_GEN_DB.CONCEPT_GEN_SCHEMA.IDEA_REPOSITORY').to_pandas()
idea = idea_df.sample()

# brand_description = idea['PRODUCT'].iloc[0]
brand_description = 'Charisma Paper Towels'
concept_idea = idea['IDEA_TEXT'].iloc[0]

In [ ]:
# OPTIONAL: Have Snowflake Cortex reexplain the concept - sometimes can provide better guidance on an image to
# a Text2Image Model
instruction_1 = f'You are developing an image for advertising the {brand_description} brand. '
instruction_2 = """Summarize the following idea into a short set of instructions that would give a 
                   Text 2 Image AI model guidance to recreate. Mention specifics about colors,
                   objects, locations of objects. Do not include any additional text outside the 
                   instructions. Do not use any bullet points or special characters except for
                   semicolons to separate instructions. Do not mention the brand name at all.
                   Do not provide JSON output, only a text string."""
instruction = instruction_1 + instruction_2

cortex_prompt = [
    {'role':'system', 'content': instruction},
    {'role':'user', 'content': concept_idea}
]
response = json.loads(Complete('llama3.1-70b', cortex_prompt))
concept_idea = response['choices'][0]['messages'].replace('"','')
    
concept_idea

## STEP 3: Generate the concept image

In [ ]:
# Create our concept
# Note: Depending on the model and the size of the compute pool you created, this can be quick or can take multiple minutes.
#       Pay attention to the status

# Generate a new background
model_output = pipeline('realistic photograph. ' + concept_idea)


# Process the image
unprocessed_img = model_output["images"][0]
array = np.array(unprocessed_img, dtype=np.uint8)
img_background = Image.fromarray(array)

# Display the image
st.image(img_background)

## STEP 4: Merge the logo/product image with the concept

In [ ]:
# Overlay the two images on top of each other, using (0,0) as the coordinate will place this image in the top-left
img_background_temp = img_background.copy()
img_background_temp.paste(img_logo, (0,0), mask=img_logo)
st.image(img_background_temp)

## STEP 5: Optionally, add a marketing tagline to an advertising concept

In [ ]:
# Create a witty tagline using a Text LLM
instruction = 'Please provide a witty advertising tagline that will be displayed at the bottom of the image described below. ' \
              'Please do not provide any additional text or descriptions, just the tagline only. If the tagline is more than ' \
              '10 words long please insert a new line character'
prompt = brand_description + ' brand with a background of ' + concept_idea
cortex_prompt = [
    {'role':'system', 'content': instruction},
    {'role':'user', 'content': prompt}
]
response = json.loads(Complete('llama3.1-70b', cortex_prompt))
tagline = response['choices'][0]['messages'].replace('"','')

tagline

In [ ]:
# Add the tagline to the concept image
width, height = img_background_temp.size

# Add a larger canvas to add a tagline at the bottom
full_concept_img = Image.new(mode='RGB', size=(width, height+100), color=(255,255,255))
full_concept_img.paste(img_background_temp, (0,0))

# Add text to the image
# tagline = 'Enjoy the holidays'
font = ImageFont.load_default(size=40)
draw = ImageDraw.Draw(full_concept_img)
_, _, w, h = draw.textbbox((0,0), tagline, font=font)
draw.text(((width-w)/2, height + ((100-h)/2)), tagline, font=font, fill='black')

In [ ]:
# Display our final concept
st.image(full_concept_img)

## APPENDIX: Additional Approaches for other use cases

Here are some other ideas that could be valuable.

1. Connecting the imageLLM to a set of consumer interviews about new ideas to quickly generate those ideas for team evaluation
2. Connecting data sets around trends or recipes or market research predictions and scheduling the model to run daily and generate various concepts that can be reviewed.